# 6 — Agglomerative (Hierarchical) Clustering on Spotify (Beginner-Friendly)

**Goal:** Explore hierarchical clustering for playlist prototyping. Compare linkages, visualize clusters, and (optionally) inspect a dendrogram.

**You’ll learn:**
- How **Agglomerative** builds a hierarchy (bottom-up merges)
- What **linkage** means (ward / average / complete) and when to use each
- How to evaluate k=20 clusters with **Silhouette / Davies–Bouldin / Calinski–Harabasz**
- How to plot the result in **PCA 2D** and (optionally) a **dendrogram** on a subsample

> Hierarchical clustering is great for **editorial workflows**: you can choose how deep to cut the tree to get your playlists.

## 0) Imports & Setup

In [ ]:

import numpy as np
import pandas as pd
import re
from pathlib import Path

import matplotlib.pyplot as plt

from sklearn.preprocessing import QuantileTransformer, StandardScaler
from sklearn.cluster import AgglomerativeClustering, KMeans
from sklearn.decomposition import PCA
from sklearn.metrics import silhouette_score, davies_bouldin_score, calinski_harabasz_score

# Optional dendrogram (requires SciPy). If missing, install via requirements and rerun.
try:
    from scipy.cluster.hierarchy import linkage, dendrogram
    SCIPY_OK = True
except Exception:
    SCIPY_OK = False

plt.rcParams['figure.figsize'] = (7,5)
RNG = np.random.RandomState(42)


## 1) Load and prepare the data
We clean column names and select Spotify audio features used across the project.

In [ ]:

DATA = Path("../data/spotify_5000_songs.csv")
assert DATA.exists(), f"Missing data at {DATA}. Place your CSV there."

def clean_col(c):
    s = re.sub(r"\s+", " ", str(c)).strip()
    return s.split(" ")[0]

df_raw = pd.read_csv(DATA)
df = df_raw.copy()
df.columns = [clean_col(c) for c in df.columns]

FEATURES = ['danceability','energy','acousticness','instrumentalness','liveness','valence',
            'tempo','speechiness','loudness','duration_ms','key','mode','time_signature']
available = [c for c in FEATURES if c in df.columns]
X = df[available].apply(pd.to_numeric, errors='coerce').dropna()

print('Using features:', available, '| Shape:', X.shape)


## 2) Scale features
Ward linkage (variance-minimizing) is particularly sensitive to scaling. We’ll default to **QuantileTransformer** (good for skew).

In [ ]:

use_quantile = True

if use_quantile:
    scaler = QuantileTransformer(output_distribution='normal', n_quantiles=min(1000, len(X)), random_state=42)
else:
    scaler = StandardScaler()

Xt = scaler.fit_transform(X)
Xt[:2]


## 3) What does *linkage* mean?
- **ward**: merges that minimize increase in within-cluster variance → favors compact, spherical clusters (K-Means-like)
- **average**: merges based on average distance between points across clusters → tolerant to shape, less sensitive to outliers than complete
- **complete**: merges using the **max** pairwise distance → tends to produce tighter, chained clusters

We’ll compare **k=20** clusters across these linkages and score them.

## 4) Evaluate linkages at **k=20**

In [ ]:

def safe_metrics(Xt, labels):
    uniq = set(labels)
    if len(uniq) < 2:
        return {'silhouette': None, 'davies_bouldin': None, 'calinski_harabasz': None}
    return {
        'silhouette': float(silhouette_score(Xt, labels)),
        'davies_bouldin': float(davies_bouldin_score(Xt, labels)),
        'calinski_harabasz': float(calinski_harabasz_score(Xt, labels)),
    }

rows = []
for link in ['ward','average','complete']:
    # ward requires Euclidean distance and is only defined for numeric features (we have that)
    model = AgglomerativeClustering(n_clusters=20, linkage=link)
    labels = model.fit_predict(Xt)
    met = safe_metrics(Xt, labels)
    rows.append({'linkage': link, **met})

df_link = pd.DataFrame(rows).sort_values('silhouette', ascending=False)
df_link


**How to read this table**
- **Higher Silhouette / Calinski–Harabasz** and **lower Davies–Bouldin** are better
- If **ward** is top: your clusters are closer to spherical (K-Means-like)
- If **average/complete** improve: your data might have elongated or chained shapes

## 5) Visualize the best linkage on a PCA 2D map
We’ll pick the best row by Silhouette and plot the clusters in 2D PCA space.

In [ ]:

# Pick the best linkage by silhouette (fallback to ward if all are None)
best_link = df_link.dropna(subset=['silhouette']).head(1)['linkage'].iloc[0] if df_link['silhouette'].notna().any() else 'ward'
print("Best linkage:", best_link)

# Fit best model
best_model = AgglomerativeClustering(n_clusters=20, linkage=best_link)
labels = best_model.fit_predict(Xt)

# 2D PCA projection for visualization only
P2 = PCA(n_components=2, random_state=42).fit_transform(Xt)
plt.scatter(P2[:,0], P2[:,1], c=labels, s=5)
plt.title(f'Agglomerative ({best_link}, k=20) — PCA 2D')
plt.xlabel('PC1'); plt.ylabel('PC2')
plt.show()


**Reading the plot**
- Tighter color islands → clearer cluster separation
- Mixed colors → consider a different linkage or number of clusters
- Compare this to your **K-Means** PCA plot — is hierarchy separating genres/moods more intuitively?

## 6) (Optional) Dendrogram on a subsample
A dendrogram shows how clusters merge **step-by-step**. It’s great for storytelling with editors.

**Note:** For speed/readability we use a **subsample** and **truncate** the tree to top levels.

In [ ]:

if SCIPY_OK:
    n = min(1500, len(Xt))  # subsample for readability/performance
    idx = RNG.choice(len(Xt), size=n, replace=False)
    Xs = Xt[idx]

    # Ward linkage for dendrogram (works nicely with scaled numeric features)
    Z = linkage(Xs, method='ward')
    plt.figure(figsize=(12, 6))
    dendrogram(Z, truncate_mode='level', p=10)  # show top 10 merge levels
    plt.title('Agglomerative (Ward) — Dendrogram (truncated)')
    plt.xlabel('Sample index/cluster'); plt.ylabel('Distance')
    plt.show()
else:
    print("SciPy not available; install scipy>=1.10 and rerun to see the dendrogram.")


## 7) Save metrics for your report

In [ ]:

OUT = Path("../reports")
OUT.mkdir(parents=True, exist_ok=True)
out_path = OUT / "agglomerative_linkage_k20.csv"
df_link.to_csv(out_path, index=False)
print(f"Saved: {out_path}")


---
## 8) Takeaways for Moosic
- **Ward** often matches K-Means in spirit (compact, centroid-like clusters)
- **Average/Complete** can surface different structures (useful if moods are chained/gradual)
- The **hierarchy** helps editors decide how granular playlists should be (cut higher → fewer playlists; cut lower → more specialized moods)

**Recommendation:** Keep both **K-Means** (fast, simple) and **Agglomerative** (explainable hierarchy) in your toolkit. Use whichever yields clearer mood islands for a given dataset.